In [17]:
# Title: A_Aug_Replace_Class_Using_HSV_Blue_colour_threshold.jpynb 
# Variant of # Title: A_Aug_Replace_Class_Using_HSV_Red_Yellow_colour_threshold.jpynb to target blue 
# Updated HSV Thresholds for Bluesimport os
import cv2
import numpy as np
import shutil
import matplotlib.pyplot as plt
from matplotlib import rcParams

# Set a different font to avoid issues with DejaVuSans
rcParams.update({'font.family': 'sans-serif', 'font.sans-serif': ['Arial']})

def fill_flag_mask(img_path, annotations, target_class_id, output_mask_path=None):
    """
    Processes an image to refine blue flag areas within a new search area based on HSV thresholds
    and fills all detected regions with sampled background colors. It visualizes sampling regions with bounding boxes.

    Args:
        img_path (str): Path to the input image.
        annotations (list): List of YOLO-format annotations for the image.
        target_class_id (int): The class ID for which the mask should be refined within the adjusted search area.
        output_mask_path (str): Path to save the processed image with filled areas.

    Returns:
        img_filled (numpy.ndarray): The processed image.
    """
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Image not found or unable to read: {img_path}")
    
    original_img = img.copy()  # Keep a copy for visualization
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Thresholds for blue in HSV (updated with extracted ranges) # These are based on the colours in specific flag images 
    lower_blue = np.array([104, 3, 163])
    upper_blue = np.array([122, 47, 236])

    # wider range 
    #lower_blue = np.array([90, 50, 50])
    #upper_blue = np.array([130, 255, 255])

    # To add more darker blues decrease the lower bound of Value (V) to 30.
    # To add lighter blues increase the upper bound of Hue (H) slightly to 140.

    #lower_blue = np.array([90, 50, 30])
    #upper_blue = np.array([130, 255, 255])

    # Generate mask for blue
    mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)

    # Refine the mask
    kernel = np.ones((3, 3), np.uint8)
    filled_mask = cv2.morphologyEx(mask_blue, cv2.MORPH_CLOSE, kernel)
    smoothed_mask = cv2.medianBlur(filled_mask, 5)

    # Locate the bounding box for the target class
    image_height, image_width = img.shape[:2]
    target_boxes = [
        annotation for annotation in annotations
        if int(annotation[0]) == target_class_id
    ]

    if not target_boxes:
        raise ValueError(f"No bounding box found for target class ID {target_class_id}.")

    # Process each target bounding box
    for box in target_boxes:
        class_id, x_center, y_center, width, height = box
        x_min = int((x_center - width / 2) * image_width)
        y_min = int((y_center - height / 2) * image_height)
        x_max = int((x_center + width / 2) * image_width)
        y_max = int((y_center + height / 2) * image_height)

        # Calculate the adjusted search area
        search_height = int(height * image_height)
        search_width = int(width * image_width)

        search_x_min = x_min
        search_x_max = x_max
        search_y_max = y_min + (y_max - y_min) // 2
        search_y_min = max(0, search_y_max - search_height)

        # Restrict the mask to the new search area
        restricted_mask = np.zeros_like(smoothed_mask)
        restricted_mask[search_y_min:search_y_max, search_x_min:search_x_max] = smoothed_mask[search_y_min:search_y_max, search_x_min:search_x_max]

        # Find contours in the restricted mask
        contours, _ = cv2.findContours(restricted_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            continue

        # Iterate through all detected contours
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)

            # Adjust sampling to only go left and up or right and up
            surrounding_regions = [
                (max(0, y-10), y, x, x+w)  # Up
            ]

            sampled_colors = []
            for (r_y_min, r_y_max, r_x_min, r_x_max) in surrounding_regions:
                region = img[r_y_min:r_y_max, r_x_min:r_x_max]
                if region.size > 0 and np.any(region):
                    sampled_colors.append(region.mean(axis=(0, 1)))

                    # Draw bounding box around sampling regions for visualization
                    cv2.rectangle(original_img, (r_x_min, r_y_min), (r_x_max, r_y_max), (0, 255, 0), 2)

            if not sampled_colors:
                print(f"Sampling failed for bounding box: x={x}, y={y}, w={w}, h={h}")
                continue

            # Calculate the average color and fill the region
            fill_color = np.mean(sampled_colors, axis=0).astype(np.uint8)
            img[y:y+h, x:x+w][restricted_mask[y:y+h, x:x+w] > 0] = fill_color

    # Save the result if output path is provided
    if output_mask_path:
        cv2.imwrite(output_mask_path, img)
    return img
    
def fill_class_by_colour(src_folder, tgt_folder, annotations_src_folder, target_class_id):
    """
    Processes all images in a folder to refine and fill flag areas based on HSV thresholds within the adjusted search area.

    Args:
        src_folder (str): Path to the folder containing source images.
        tgt_folder (str): Path to the folder to save processed images.
        annotations_src_folder (str): Path to the folder containing YOLO annotations.
        target_class_id (int): The class ID for which the mask should be refined within the adjusted search area.
    """
    if not os.path.exists(tgt_folder):
        os.makedirs(tgt_folder)

    for file_name in os.listdir(src_folder):
        if file_name.lower().endswith('.png'):
            src_image_path = os.path.normpath(os.path.join(src_folder, file_name))
            annotation_path = os.path.normpath(os.path.join(annotations_src_folder, os.path.splitext(file_name)[0] + ".txt"))
            
            if not os.path.exists(annotation_path):
                print(f"No annotation file for {file_name} in {src_folder}. Skipping.")
                continue

            with open(annotation_path, 'r') as f:
                annotations = [list(map(float, line.split())) for line in f]

            intermediate_mask_path = os.path.normpath(os.path.join(tgt_folder, f"{file_name}"))
            fill_flag_mask(src_image_path, annotations, target_class_id, intermediate_mask_path)

            # Copy the annotation file to the target folder
            target_annotation_path = os.path.normpath(os.path.join(tgt_folder, os.path.basename(annotation_path)))
            shutil.copy(annotation_path, target_annotation_path)




# Run to fill target area with bg colour 
src_folder = 'D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/Merge_red_yellow_mask_from_class_1_in_30_to_24/000328_Filled2'
tgt_folder = 'D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/Merge_red_yellow_mask_from_class_1_in_30_to_24/000328_Filled3'
annotations_src_folder = 'D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/Merge_red_yellow_mask_from_class_1_in_30_to_24/000328'
target_class_id = 1 # Red Yellow flag 3  # Class ID to target = 3 Green Cost flag 

fill_class_by_colour(src_folder, tgt_folder, annotations_src_folder, target_class_id)
